# HW5-3: CIFAR image classification using VGG16 or VGG19 with pretrained weights (implemented in tf.keras or PyTorch Lightning).

- The code in this notebook was generated using [GitHub Copilot](https://github.com/features/copilot).

## Prompt

Generate a PyTorch Lightning script for CIFAR-10 classification using:

- Model: Pre-trained VGG16 from ImageNet. Replace the final layer to classify 10 CIFAR-10 classes.
- Loss: Cross-entropy.
- Metrics: Track training, validation, and test accuracy using torchmetrics.
- Optimizer: AdamW with CosineAnnealingLR scheduler.
- Data: CIFAR-10 with pre-trained model transforms.
- Logger: TensorBoardLogger.
- Callbacks: LearningRateMonitor, TQDMProgressBar.
- Training: Train for 10 epochs, batch size 128, learning rate 1e-4.

In [1]:
%load_ext tensorboard

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets, models
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor, TQDMProgressBar
import torchmetrics

class VGG16Model(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):
        super().__init__()
        self.learning_rate = learning_rate

        # Load the pre-trained VGG16 model and modify it
        self.model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        
        # freeze all layers
        # for i, param in enumerate(self.model.features.parameters()):
        #     param.requires_grad = False
        
        # Modify the final fully connected layer for 10 output classes (CIFAR-10)
        self.model.classifier[6] = nn.Linear(4096, 10)

        # Metrics
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = self.train_acc(logits.softmax(dim=-1), y)
        self.log("train_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("train_acc", acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = self.val_acc(logits.softmax(dim=-1), y)
        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("val_acc", acc, prog_bar=True, on_step=False, on_epoch=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = self.test_acc(logits.softmax(dim=-1), y)
        self.log("test_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("test_acc", acc, prog_bar=True, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.trainer.max_epochs)
        return [optimizer], [scheduler]


# Data preparation for CIFAR-10
def prepare_data():
    transform = models.VGG16_Weights.IMAGENET1K_V1.transforms()
    dataset = datasets.CIFAR10(
        root="data", train=True, download=True, transform=transform
    )
    cifar10_test = datasets.CIFAR10(
        root="data", train=False, download=True, transform=transform
    )
    cifar10_train, cifar10_val = random_split(dataset, [45000, 5000])
    return cifar10_train, cifar10_val, cifar10_test


cifar10_train, cifar10_val, cifar10_test = prepare_data()
train_loader = DataLoader(cifar10_train, batch_size=128, shuffle=True)
val_loader = DataLoader(cifar10_val, batch_size=128)
test_loader = DataLoader(cifar10_test, batch_size=128)

# Logger and Callbacks
logger = TensorBoardLogger(
    "logs5-3",
    name="cifar10_vgg16",
)
lr_monitor = LearningRateMonitor(logging_interval="epoch")
progress_bar = TQDMProgressBar(leave=True)

# Train the model
model = VGG16Model(learning_rate=1e-4)
trainer = pl.Trainer(
    max_epochs=10,
    logger=logger,
    callbacks=[lr_monitor, progress_bar],
    log_every_n_steps=10,
)
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)


Files already downloaded and verified
Files already downloaded and verified


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2024-12-17 04:06:13.068298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 04:06:13.078160: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 04:06:13.081077: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 04:06:13.088538: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other opera

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/g113056077/.pyenv/versions/aiot-hw5/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


/home/g113056077/.pyenv/versions/aiot-hw5/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]
Epoch 9: 100%|██████████| 352/352 [03:58<00:00,  1.47it/s, v_num=0, val_loss=0.346, val_acc=0.939, train_loss=0.000216, train_acc=1.000]


`Trainer.fit` stopped: `max_epochs=10` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/g113056077/.pyenv/versions/aiot-hw5/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 79/79 [00:17<00:00,  4.57it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9333000183105469     │
│         test_loss         │    0.36675041913986206    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.36675041913986206, 'test_acc': 0.9333000183105469}]

In [3]:
model

VGG16Model(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16

In [4]:
%tensorboard --logdir "logs5-3"